In [13]:
#from classes.p123api import p123api
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
# from classes.p123login import p123api
import pandas as pd
import datetime
import pickle
from time import sleep
import time
import os
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options


In [7]:
loginUrl = 'https://www.portfolio123.com/app/auth'
appOpener = 'https://www.portfolio123.com/app/dashboard'

chrome_location = "C:/chromedriver_win32/chromedriver.exe"
# os.chdir(r"C:\Users\enisbe\Google Drive\portfolio123\screen_data\short_interest")
save_location = "C:/Users/enisb/python/nw_python/p123stock-project"
 
project_location ="C:/Users/enisb/python/nw_python/p123stock-project"# win
downloads = f'{project_location}/data/download'
downloads

'C:/Users/enisb/python/nw_python/p123stock-project/data/download'

In [14]:

def get_browser(set_download_location = ""):
    download_folder = set_download_location
    # chromedriver_path = "/path/to/chromedriver"
    chromedriver_path = "C:/chromedriver_win32/chromedriver.exe"

    chromeService = Service(s=chrome_location)
    chromeOptions = webdriver.ChromeOptions()
    # chromeOptions.add_argument("--headless")    
    # chromeOptions.add_argument("--no-sandbox")
    # chromeOptions.add_argument("--disable-dev-shm-usage")
    prefs = {"profile.default_content_settings.popups": 0,
            "download.default_directory" : download_folder,
             "download.prompt_for_download": False,
            "download.directory_upgrade": True
            }
    chromeOptions.add_experimental_option("prefs",prefs)
    if download_folder == "": 
        browser = webdriver.Chrome(service =chromeService)
    else:
        browser = webdriver.Chrome(service =chromeService, options=chromeOptions)
    
    browser.set_page_load_timeout(180)
    return browser

def p123_login(browser):

    browser.get(loginUrl)
    try:
        
        with open(f'{project_location}/selenium/cookie3', 'rb') as cookiesfile:
            cookies = pickle.load(cookiesfile)
            for cookie in cookies:
                browser.add_cookie(cookie)
    except:
        print("No Cookie file")
        
    try:
        browser.get(appOpener)
        time.sleep(2)

        assert 'Dashboard - Portfolio123' in browser.title
        print("LOGGED IN WITH COOKIES")

    except:
        browser.get(loginUrl)
        browser.find_element(By.ID,'user').send_keys('enisbe')
        browser.find_element(By.ID,'passwd').send_keys('sarajevo76')
        browser.find_element(By.CSS_SELECTOR,'.ng-binding').click()

        time.sleep(2)
        with open(f'{project_location}/selenium/cookie3', 'wb') as filehandler:
            pickle.dump(browser.get_cookies(), filehandler)


In [9]:
def get_downloaded_ranks(download_path):
    if download_path:
        already_downloaded = os.listdir(download_path)
    else: 
        already_downloaded = os.listdir()


    if '.ipynb_checkpoints' in already_downloaded:    
        already_downloaded.remove('.ipynb_checkpoints')
    
    data_downloaded = []
    downloaded_ranks=set()
    for file in already_downloaded:

        date_report  = file.split('_')

        row =  {"rank_date": date_report[0], 'start_date': date_report[1],'end_date':date_report[2], 'report': date_report[3] + date_report[4]}
        data_downloaded.append(row)
        downloaded_ranks=set(pd.DataFrame(data_downloaded)['rank_date'])


    return downloaded_ranks
#Step 1: Pull the screen table

def get_screen_table():
    row_two = browser.find_elements(By.XPATH,'//*[@id="results-table"]/table/thead/tr[2]')[0]

    headers = [col.text for col in row_two.find_elements(By.XPATH, './/th')]
    headersClean = [w.replace('\n', '_').replace(' ', '_').replace('*', '') for w in headers]


    table_body = browser.find_elements(By.XPATH,'//*[@id="results-table"]/table/tbody')[0]

    table = []
    for row in table_body.find_elements(By.XPATH,'.//tr'):
        row_list = []
        try:
            link = row.find_element(By.TAG_NAME,"a")
        except:
            link = None
        row_list.append(link)
        for cell in row.find_elements(By.XPATH,'.//td'):
            row_list.append(cell.text)

        table.append(row_list)

    headersClean.insert(0,"link")
    result_table = pd.DataFrame(table,columns=headersClean)
    return  result_table


def is_saved_report_selected(parent):
    # Just so that we dont have to switch between report type we will check if rerport is already selected
    return parent.find_element(By.ID, "radioSaved").is_selected()
    
# Select All Stocks
"""
the trick and understand select All is select the text in drop down will be 20 the number of stocks selected will be
all. So if you want to go back to 20 you must select 50 first (which updates for some reason) and then 20.
"""

def is_max_stock_option_selected(parent):
    selected_stocks = parent.find_element(By.XPATH, '//*[@id="run-section"]/div[1]/div[1]/div/span')
    selected_stock = selected_stocks.text.split()
    max_selected = int(selected_stock[4])
    cur_selected = int(selected_stock[2])
    all_selected = max_selected==cur_selected
    return all_selected


def selected_report(parent):
    all_reports = parent.find_elements(By.XPATH,'//*[@id="saved-report-list"]/ul')[0]
    report_selected = all_reports.find_elements(By.XPATH,"//li[contains(@class,'selected')]")
 
    if report_selected: 
        report = report_selected[0].text
    else:
        report = None
    return report


def run_backtest(backtest, **kwargs):
    screens = {'testvalue':"https://www.portfolio123.com/app/screen/summary/229509?st=7&mt=1"}

    url = screens[backtest]
    if url != browser.current_url:
        browser.get(screens[backtest])

    backtest_tab = browser.find_element(By.XPATH, '//*[@id="scrtab_3"]')
    backtest_tab.click()


    clear_button = browser.find_element(By.XPATH, '//*[@id="clearResults"]')
    clear_button_class = clear_button.get_attribute('class')
    button_disabled =  'disabled' in clear_button_class
    if not button_disabled:
        try:
            clear_button.click()
            sleep(2)
        except:
            print("something wrong with clear clear results")

    run_backtest = browser.find_element(By.XPATH, '// *[ @ id = "runBacktest"]')
    class_backtest = run_backtest.get_attribute('class')
    sleep(1)
    backtest_disabled = 'disabled' in class_backtest
    if backtest_disabled:
        print("Backtest Button Disabled. Check the problem")
    else:
        run_backtest.click()
    sleep(5)

In [78]:

def loop_report_download():
    print('Execute: loop_report_download()')
    print('-' * 50)
    # print('-' * 10)
    print("STEP: 1")
    print('-' * 10)
    print("Describe: Are we in corect location after click the link?")

    try:
        search_text = "//*[contains(text(),'Rank data as of {}, Start Date: {}, End Date: {}')]".format(get_rankdt,
                                                                                                        get_trandt,
                                                                                                        get_enddt)
        screen_divtag = browser.find_elements(By.XPATH, search_text)[0]

    except:
        search_text = "//*[contains(text(),'Rank data as of {}')]".format(get_rankdt)
        screen_divtag = browser.find_elements(By.XPATH, search_text)[0]

    parent = screen_divtag.find_elements(By.XPATH, '././..')[0]

    if parent.get_attribute('id') == 'dia':
        print("correctly selected element")
    else:
        print("not right table element")

    print("Sleep 2 sec after check for the correct location'")
    sleep(2)
    print('-' * 10)
    print("STEP: 2")
    print('-' * 10)
    print("Describe: Are we in the saved reports section? If not change")
    #check if saved report section is selected
    saved_reports = is_saved_report_selected(parent)
    
    if not saved_reports:
        try:
            saved_reports = parent.find_element(By.ID, "radioSaved")

            browser.execute_script("arguments[0].click();", saved_reports)
        except:
            print("error in clicking saved report. Debug")
    else:
        print("Saved Reports already selected")
   
    print("Sleep 2 sec after check for 'Saved Reports'")
    sleep(2)

    """
    Must select max stock
    """
    print('-' * 10)
    print("STEP: 3")
    print('-' * 10)
    print("Describe: Check of the max stock is selected")

    max = True
    if max:
        print("this will take a minute or to change ")
        if not is_max_stock_option_selected(parent):
            print("Max stock is not selected. Selecting the option. Wait a little...")
            stock_number = Select(parent.find_element(By.XPATH, '//*[@id="resultrowspp"]'))

            stock_number.select_by_visible_text('All')
        else:
            print("All Stocks are selected")

    print("Sleep before moving formward with max selection")
    sleep(2)

    print('-' * 10)
    print("STEP: 4")
    print('-' * 10)
    print("Describe: Loop through the reports you want to download")

    report_to_downloand = ["Earnings_industry"] #  ['642_report', '643_report']
   
    for report in range(len(report_to_downloand)):

        current_report = selected_report(parent)
        print('-'*10)
        print("STEP: 5 - loop #", report)
        print('-'*10)
        print("Describe: Is the currently selected report the report in the list to download?")
        print("Cont: if yes the continue with the selection.")
        print("Cont: if no change to first report from the list.")

        if not current_report in report_to_downloand:
            save_report = report_to_downloand[0]
            report_to_downloand.remove(save_report)  # delete from the list for the next iteration
            print("The report {} is not selected. Selecting. Wait a little...".format(save_report))
            screen_type = "//ul/li[contains(text(),'{}')]".format(save_report)
            screen_report = parent.find_element(By.XPATH, screen_type)
            screen_report.click()
            sleep(5)
        else:
            save_report = selected_report(parent)
            report_to_downloand.remove(save_report)

        print("Currently selected report: ",  selected_report(parent))
        print("Sleep 2 sec after selecting correct report.")
        sleep(5)
        print('-' * 10)
        print("STEP 6: - loop")
        print('-' * 10)
        print("Describe: Download and save")
        parent_body = parent.find_elements(By.XPATH, ".//table")[0]

        
        
        file_name = f'{get_rankdt.replace("/", "")}_{get_trandt.replace( "/", "")}_{get_enddt.replace("/", "")}_report_{save_report.lower().replace("_","")}.pkl'
        file_path = os.path.join(downloads,  "screen_data", file_name)
       
        with open(file_path, 'wb') as output:
            pickle.dump(parent_body.get_attribute("outerHTML"), output, pickle.HIGHEST_PROTOCOL)

        print("after the table is extracted swtich back to 20 pp")
        if  is_max_stock_option_selected(parent):
            stock_number = Select(parent.find_element(By.XPATH, '//*[@id="resultrowspp"]'))
            stock_number.select_by_visible_text('20 pp')

            
        rng_min, rng_max = 5,10
        print("After downloading: "+ save_report + " random sleep between {} and {} second".format(rng_min,rng_max))
        random_sleep = np.random.randint(rng_min,rng_max)

        sleep(random_sleep)

    print("Sleep 2 Before exiting the rank_dt download section and moving to the next")
    sleep(2)
    but = browser.find_element(By.CLASS_NAME, "ui-dialog-titlebar-close")
    # but = p123.browser.find_element_by_xpath("/html/body/div[9]/div[1]/button")
    sleep(2)
    # p123.browser.execute_script("arguments[0].click();", but)
    but.click()
    sleep(2)

In [121]:
browser = get_browser(downloads)
p123_login(browser)

In [130]:
run_backtest('testvalue')
print("Sleeping 5 secs")
sleep(15)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00BF8893+48451]
	(No symbol) [0x00B8B8A1]
	(No symbol) [0x00A95058]
	(No symbol) [0x00A7D073]
	(No symbol) [0x00ADDEBB]
	(No symbol) [0x00AEBFD3]
	(No symbol) [0x00ADA0B6]
	(No symbol) [0x00AB7E08]
	(No symbol) [0x00AB8F2D]
	GetHandleVerifier [0x00E58E3A+2540266]
	GetHandleVerifier [0x00E98959+2801161]
	GetHandleVerifier [0x00E9295C+2776588]
	GetHandleVerifier [0x00C82280+612144]
	(No symbol) [0x00B94F6C]
	(No symbol) [0x00B911D8]
	(No symbol) [0x00B912BB]
	(No symbol) [0x00B84857]
	BaseThreadInitThunk [0x76AF7D59+25]
	RtlInitializeExceptionChain [0x7749B74B+107]
	RtlClearBits [0x7749B6CF+191]


In [123]:
result_table =get_screen_table()

In [20]:
# get_downloaded_ranks(f"{downloads}/screen_data")

In [125]:
all_ranks  = set(result_table['Rank_Dt'].apply(lambda x: x.replace("/","")).unique())
result_table = result_table.dropna() #apply(lamda x: if x is None ) # .reset_index()
downloaded_ranks=  get_downloaded_ranks(f"{downloads}/screen_data")
result_table_to_download = result_table[result_table['Rank_Dt'].str.replace("/","").isin( list(all_ranks - downloaded_ranks))]


In [126]:
downloaded = []


In [127]:
max_iterations = 5
iter = np.min([max_iterations, result_table_to_download.shape[0]])

In [129]:
 
print("Will iterate: ", iter, "x times")


for i in range(0, iter): #  result_table_to_download.shape[0]):

    try: 
        start = time.time()
        rw = result_table_to_download.sample(n=1)
        downloading_rank_dt = rw['Rank_Dt'].iloc[0]
        print('-'*100)
        print( i + 1 , "Download ranks as of: ", downloading_rank_dt)
        print('-'*100)
    
        downloaded.append(downloading_rank_dt)
    
        get_link = rw['link'].iloc[0]
        get_rankdt =  rw['Rank_Dt'].iloc[0]
        get_trandt = rw['Tran_Dt'].iloc[0]
        get_enddt = rw['End_Dt'].iloc[0]
        get_link.click()
    
        print("Wait 5 seconds before executing loop_report_download()...")
        sleep(5)
    
    
        loop_report_download()
    
    
        print("Finished with {} loop".format(i))
    
        result_table_to_download= result_table_to_download[result_table_to_download['Rank_Dt'] != downloading_rank_dt]
        random_sleep = np.random.randint(60, 300)
        # random_sleep = np.random.randint(2, 15)
        end = time.time()
        print("Iteration Time: ", end-start)
        print("End Iteration: ", str(i+1), " Sleep: ", random_sleep , ' sec')
    
        if i+1 < iter:
            sleep(random_sleep) 
    except:
        run_backtest('testvalue')
        print("Sleeping 5 secs")
        sleep(15)
        result_table =get_screen_table()
        all_ranks  = set(result_table['Rank_Dt'].apply(lambda x: x.replace("/","")).unique())
        result_table = result_table.dropna() #apply(lamda x: if x is None ) # .reset_index()
        downloaded_ranks=  get_downloaded_ranks(f"{downloads}/screen_data")
        result_table_to_download = result_table[result_table['Rank_Dt'].str.replace("/","").isin( list(all_ranks - downloaded_ranks))]


Will iterate:  5 x times
----------------------------------------------------------------------------------------------------
1 Download ranks as of:  12/23/17
----------------------------------------------------------------------------------------------------


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00BF8893+48451]
	(No symbol) [0x00B8B8A1]
	(No symbol) [0x00A95058]
	(No symbol) [0x00A7D073]
	(No symbol) [0x00ADDEBB]
	(No symbol) [0x00AEBFD3]
	(No symbol) [0x00ADA0B6]
	(No symbol) [0x00AB7E08]
	(No symbol) [0x00AB8F2D]
	GetHandleVerifier [0x00E58E3A+2540266]
	GetHandleVerifier [0x00E98959+2801161]
	GetHandleVerifier [0x00E9295C+2776588]
	GetHandleVerifier [0x00C82280+612144]
	(No symbol) [0x00B94F6C]
	(No symbol) [0x00B911D8]
	(No symbol) [0x00B912BB]
	(No symbol) [0x00B84857]
	BaseThreadInitThunk [0x76AF7D59+25]
	RtlInitializeExceptionChain [0x7749B74B+107]
	RtlClearBits [0x7749B6CF+191]


In [132]:
#!/usr/bin/env python
import psutil
# gives a single float value
print(psutil.cpu_percent())
# gives an object with many fields
print(psutil.virtual_memory())
# you can convert that object to a dictionary 
print(dict(psutil.virtual_memory()._asdict()))
# you can have the percentage of used RAM
print(psutil.virtual_memory().percent)
 
# you can calculate percentage of available memory
print(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)
 

1.5
svmem(total=29734412288, available=17646145536, percent=40.7, used=12088266752, free=17646145536)
{'total': 29734412288, 'available': 17647312896, 'percent': 40.7, 'used': 12087099392, 'free': 17647312896}
40.6
59.35114551136475
